In [0]:
import requests


In [0]:
CSV_URL = (
    "https://docs.google.com/spreadsheets/d/1reziwqueMdfNA-3Vgz-JxVK_XXaB99nuOvHSdC0wBnw/export?format=csv&gid=919246905"
   # "https://docs.google.com/spreadsheets/d/1reziwqueMdfNA-3Vgz-JxVK_XXaB99nuOvHSdC0wBnw/edit?usp=sharing"
   #"https://docs.google.com/spreadsheets/d/1reziwqueMdfNA-3Vgz-JxVK_XXaB99nuOvHSdC0wBnw/edit?gid=919246905#gid=919246905"
)
dbfs_path = "/Volumes/transacional/case_gocase/tmp/Case Dados - Pedidos.csv"

response = requests.get(CSV_URL)
response.raise_for_status()

with open(dbfs_path, "w", encoding="utf-8") as f:
    f.write(response.text)



In [0]:
CSV_URL = (
    "https://docs.google.com/spreadsheets/d/1c7uf9pHUN5JvNJHuueXLkaGkX8vvslWd5-NiPYoZ8V0/export?format=csv&gid=224192872"
    #"https://docs.google.com/spreadsheets/d/1c7uf9pHUN5JvNJHuueXLkaGkX8vvslWd5-NiPYoZ8V0/edit?usp=sharing"
    #https://docs.google.com/spreadsheets/d/1c7uf9pHUN5JvNJHuueXLkaGkX8vvslWd5-NiPYoZ8V0/edit?gid=1760558779#gid=1760558779
)
dbfs_path = "/Volumes/transacional/case_gocase/tmp/Business Case Dados Supply.csv"

response = requests.get(CSV_URL)
response.raise_for_status()

with open(dbfs_path, "w", encoding="utf-8") as f:
    f.write(response.text)

In [0]:
CSV_URL = (
    "https://docs.google.com/spreadsheets/d/1c7uf9pHUN5JvNJHuueXLkaGkX8vvslWd5-NiPYoZ8V0/export?format=csv&gid=1760558779"
    #"https://docs.google.com/spreadsheets/d/1c7uf9pHUN5JvNJHuueXLkaGkX8vvslWd5-NiPYoZ8V0/edit?usp=sharing"
    #https://docs.google.com/spreadsheets/d/1c7uf9pHUN5JvNJHuueXLkaGkX8vvslWd5-NiPYoZ8V0/edit?gid=1760558779#gid=1760558779
)
dbfs_path = "/Volumes/transacional/case_gocase/tmp/Business Case Dados Itens.csv"

response = requests.get(CSV_URL)
response.raise_for_status()

with open(dbfs_path, "w", encoding="utf-8") as f:
    f.write(response.text)

Análise do Desafio 1: Oscilações nas Vendas e Impacto de Promoções
Objetivo: Entender o comportamento das vendas ao longo do tempo e medir se promoções como frete grátis ou queima de estoque (stock_burning) realmente funcionam.

In [0]:
#Vamos visualizar o faturamento e o número de pedidos por dia para identificar picos, quedas e padrões.
#O que analisar nos resultados:
# Picos de Vendas: Existem dias específicos (finais de semana, início/fim de mês) com vendas muito mais altas?
# Sazonalidade: Você consegue observar algum padrão semanal ou mensal?
# Visualização: Use a função de plotagem do Databricks para transformar essa tabela em um gráfico de linhas. Isso tornará as oscilações muito mais evidentes.
# from pyspark.sql.functions import sum, count, col, date_trunc, avg, when

# Usamos o df_pedidos para garantir que cada pedido seja contado apenas uma vez.
vendas_diarias = df_pedidos.groupBy("created_at_data") \
                           .agg(
                               sum("valor_nf").alias("faturamento_total"),
                               count("order_id").alias("numero_pedidos")
                           ).orderBy("created_at_data")

print("--- Vendas Diárias (Faturamento e N° de Pedidos) ---")
display(vendas_diarias)

In [0]:
# Queremos saber se oferecer frete grátis incentiva os clientes a gastarem mais ou a comprarem mais itens.
# O que analisar nos resultados:

# Ticket Médio: O valor médio do pedido (ticket_medio) é significativamente maior nos pedidos com "Frete Grátis"? Se sim, a promoção está se pagando.

# Média de Itens: Os clientes adicionam mais produtos ao carrinho para atingir o frete grátis? Compare a media_itens_por_pedido.
# Segmentamos os pedidos em dois grupos: com frete pago e com frete grátis.
analise_frete = df_pedidos.withColumn("tipo_frete", when(col("frete_cliente") > 0, "Pago").otherwise("Grátis")) \
                          .groupBy("tipo_frete") \
                          .agg(
                              avg("valor_nf").alias("ticket_medio"),
                              avg("num_itens").alias("media_itens_por_pedido"),
                              count("order_id").alias("total_pedidos")
                          )

print("--- Análise de Impacto do Frete Grátis ---")
display(analise_frete)

In [0]:
# Vamos identificar quais produtos são mais vendidos quando marcados como stock_burning.
# O que analisar nos resultados:

# Top Produtos: Quais são os itens que mais se beneficiam dessa promoção? Isso pode ajudar a decidir quais produtos colocar em queima de estoque no futuro.
# Usamos o df_vendas, pois a informação de stock_burning é a nível de item.
produtos_queima_estoque = df_vendas.filter(col("stock_burning") == True) \
                                   .groupBy("material_name", "material_category") \
                                   .agg(count("*").alias("unidades_vendidas")) \
                                   .orderBy(col("unidades_vendidas").desc())

print("--- Produtos Mais Vendidos em Queima de Estoque ---")
display(produtos_queima_estoque.limit(20))

In [0]:

# --- Ranking de Faturamento por Categoria ---
faturamento_por_categoria = df_vendas.groupBy("material_category") \
                                     .agg(round(sum("price"), 2).alias("faturamento_total"),
                                         count("order_id").alias("unidades_vendidas")
                                     ).orderBy(desc("faturamento_total"))

print("--- Ranking de Faturamento por Categoria de Produto ---")
display(faturamento_por_categoria)

# --- Ranking de Faturamento por Produto ---

faturamento_por_produto = df_vendas.groupBy("material_name") \
                                   .agg(round(sum("price"), 2).alias("faturamento_total"),
                                       count("order_id").alias("unidades_vendidas")
                                   ).orderBy(desc("faturamento_total"))

print("\n--- Top 20 Produtos por Faturamento ---")
display(faturamento_por_produto.limit(20))

Análise do Desafio 2: Rupturas de Estoque
Objetivo: Identificar quais produtos estão em falta e, mais importante, qual o impacto potencial disso na receita.

In [0]:
#Primeiro, listamos os produtos que deveriam estar à venda, mas estão com estoque zerado.
produtos_ruptura = df_supply.filter(
    (col("quantity") == 0) &
    (col("should_sell") == True) &
    (col("discontinued") == False)
).select("material_id", "material_name", "leadtime")

print(f"Total de produtos em ruptura: {produtos_ruptura.count()}")
print("--- Amostra de Produtos em Ruptura de Estoque ---")
display(produtos_ruptura.limit(10))

In [0]:
# Um produto em ruptura só é um problema se ele vende bem. Vamos cruzar a lista de ruptura com o histórico de vendas para encontrar os casos mais críticos.
# O que analisar nos resultados:

# Lista de Prioridades: O topo desta tabela (ruptura_critica) são os produtos que você precisa repor com mais urgência. Eles estão em falta e têm uma alta demanda histórica.

# leadtime vs. vendas_historicas: Um produto com vendas altíssimas e um leadtime (tempo de reposição) longo é um grande risco para a receita.
# Contar as vendas históricas para cada material_id
demanda_historica = df_vendas.groupBy("material_id") \
                             .agg(count("order_id").alias("vendas_historicas"))

# Juntar a lista de ruptura com a demanda para priorizar
ruptura_critica = produtos_ruptura.join(demanda_historica, "material_id", "inner") \
                                  .orderBy(col("vendas_historicas").desc())

print("--- Produtos Mais Críticos para Reposição (Ruptura vs. Demanda) ---")
display(ruptura_critica)

In [0]:


# 1. Contar todos os pedidos por 'status_pedido'
# O nome da variável foi alterado para 'status_count' para maior clareza
status_count = df_pedidos.groupBy("status_pedido") \
                         .agg(count("order_id").alias("total_pedidos"))

print("--- Contagem de Pedidos por Status ---")
display(status_count)


# 2. Isolar os itens de pedidos que foram efetivamente cancelados.
# (Certifique-se de usar o nome exato que aparecer na tabela acima, ex: "canceled")
status_cancelado = "canceled" # <-- AJUSTE AQUI se o nome for diferente
pedidos_cancelados = df_vendas.filter(col("status_pedido") == status_cancelado)


# 3. Cruzar os itens cancelados com a lista de produtos em ruptura de estoque.
itens_cancelados_vs_ruptura = pedidos_cancelados.join(
    produtos_ruptura, # DataFrame da Análise 2.1
    "material_id",
    "inner"
).groupBy(pedidos_cancelados["material_name"]) \
 .agg(count("*").alias("cancelamentos_de_itens_em_ruptura")) \
 .orderBy(col("cancelamentos_de_itens_em_ruptura").desc())

print(f"\n--- Itens em Ruptura que foram Cancelados (Status = '{status_cancelado}') ---")
display(itens_cancelados_vs_ruptura)

total_pedidos = df_pedidos.count()
pedidos_cancelados_count = df_pedidos.filter(col("status_pedido") == status_cancelado).count()
taxa_cancelamento_percentual = (pedidos_cancelados_count / total_pedidos) * 100

print(taxa_cancelamento_percentual)

In [0]:


# --- Passo 1: Defina o nome exato do status de cancelamento ---
# (Verifique na tabela da análise anterior se é "canceled", "Cancelado", etc.)
status_cancelado = "canceled" 

# --- Passo 2: Calcule o número total de pedidos ---
total_pedidos = df_pedidos.count()

# --- Passo 3: Calcule o número de pedidos com o status de cancelamento ---
pedidos_cancelados_count = df_pedidos.filter(col("status_pedido") == status_cancelado).count()

# --- Passo 4: Calcule e exiba a taxa de cancelamento ---
print("--- Cálculo da Taxa de Cancelamento ---")

# Verificação para evitar divisão por zero, caso o DataFrame esteja vazio
if total_pedidos > 0:
    taxa_cancelamento_percentual = (pedidos_cancelados_count / total_pedidos) * 100
    
    print(f"Total de pedidos no DataFrame: {total_pedidos}")
    print(f"Total de pedidos cancelados ('{status_cancelado}'): {pedidos_cancelados_count}")
    print(f"Taxa de cancelamento: {taxa_cancelamento_percentual:.2f}%")
else:
    print("Não há pedidos para calcular a taxa.")

In [0]:


# --- Passo 1: Calcular o total de cancelamentos para CADA item ---
# Filtramos o df_vendas para pegar apenas os itens de pedidos cancelados
# e depois contamos quantas vezes cada 'material_name' aparece.

status_cancelado = "canceled" # Confirme se este é o nome correto do status

cancelamentos_por_item = df_vendas.filter(col("status_pedido") == status_cancelado) \
                                  .groupBy("material_name") \
                                  .agg(count("*").alias("total_cancelado"))

# --- Passo 2: Preparar a lista de produtos em ruptura para o join ---
# Vamos apenas selecionar a coluna que precisamos e adicionar uma "flag"
# para indicar que esses itens estão em ruptura.

info_ruptura = produtos_ruptura.select(
    "material_name",
    lit(True).alias("em_ruptura") # lit() cria uma coluna com um valor literal (constante)
)

# --- Passo 3: Criar o DataFrame final combinando as informações ---
# Usamos um LEFT JOIN para garantir que todos os itens cancelados apareçam na lista,
# mesmo que eles NÃO estejam em ruptura de estoque.

df_analise_cancelamento = cancelamentos_por_item.join(
    info_ruptura,
    "material_name", # A chave da junção
    "left"
).na.fill(False, ["em_ruptura"]) \
 .orderBy(desc("total_cancelado")) # Ordena para ver os mais problemáticos primeiro

# --- Passo 4: Exibir o resultado ---
print("--- Análise de Cancelamentos por Item vs. Ruptura de Estoque ---")
display(df_analise_cancelamento)

Análise do Desafio 3: Problemas Logísticos
Objetivo: Medir a eficiência da entrega, identificar atrasos e comparar o desempenho de transportadoras e regiões

In [0]:
# Vamos medir o tempo que um pedido leva para sair do armazém e o tempo que a transportadora leva para entregá-lo.



# Calculamos a diferença em dias entre as etapas do pedido
df_pedidos_com_prazos = df_pedidos.withColumn("tempo_despacho_dias", datediff(col("data_envio"), col("created_at_data"))) \
                                 .withColumn("tempo_transporte_dias", datediff(col("data_entrega"), col("data_envio")))

# Calcular médias gerais
medias_logisticas = df_pedidos_com_prazos.agg(
    avg("tempo_despacho_dias").alias("media_dias_para_despacho"),
    avg("tempo_transporte_dias").alias("media_dias_em_transporte")
)

print("--- Médias de Tempo do Ciclo Logístico ---")
display(medias_logisticas)

In [0]:
# Estamos cumprindo os prazos? Qual transportadora é mais rápida e qual tem mais atrasos?
# O que analisar nos resultados:

# Ranking de Transportadoras: Qual transportadora tem o menor tempo_medio_entrega e o menor percentual_atrasos? Isso pode ser crucial para renegociar contratos.

# Análise por Estado: Você pode facilmente adaptar a consulta acima para fazer um groupBy("Estado") em vez de groupBy("Transportadora") e descobrir quais estados têm os maiores prazos ou taxas de atraso.
# Criar uma flag para pedidos atrasados
df_pedidos_com_atraso = df_pedidos_com_prazos.withColumn(
    "atrasado",
    when(col("data_entrega") > col("prazo_entrega_transportadora"), 1).otherwise(0)
)

# Agrupar por transportadora para analisar o desempenho
desempenho_transportadora = df_pedidos_com_atraso.groupBy("Transportadora") \
    .agg(
        avg("tempo_transporte_dias").alias("tempo_medio_entrega"),
        (sum("atrasado") / count("order_id") * 100).alias("percentual_atrasos"),
        count("order_id").alias("total_entregas")
    ).orderBy(col("tempo_medio_entrega"))

print("--- Desempenho por Transportadora ---")
display(desempenho_transportadora)